# <b> 1. LSA(Latent Semantic Analysis)</b> 

In [1]:
import nltk
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from nltk.tokenize import TreebankWordTokenizer

In [2]:
df_2024 = pd.read_csv('../드라마영화예능_전처리데이터/24드라마_전처리.csv')
df_2023 = pd.read_csv('../드라마영화예능_전처리데이터/23드라마_전처리.csv')
df_2022 = pd.read_csv('../드라마영화예능_전처리데이터/22드라마_전처리.csv')

df = pd.concat([df_2024, df_2023, df_2022], axis=0).reset_index(drop=True)

df.head()

,postdate,body,description,title,preprocessed_body,preprocessed_description,preprocessed_title
0,20240510,안녕하세요 오늘은 평소와 다르게 중국 드라마에 대해 설명 드리려고 합니다 치아문단순...,직접 b2024년b 곧 b방영b 예정인 b드라마b 네 작품을 가지고 왔습니다 b20...,중국 드라마 추천 2024년 방영 예정작 4편 로맨스 학원물편,은 평소 중국 설명 치아 문 단순 적소 미 호 유성화원 우리나라 히트 인하 중국 관...,기대작 중국 환우 보기 부 작,중국 작 편 로맨스 학원물 편
1,20240112,MBC에 이어 SBS도 2024년 드라마 라인업을 공개했습니다 2023 SBS 연기...,MBC에 이어 SBS도 b2024년b b드라마b 라인업을 공개했습니다 2023 SB...,2024년 SBS 드라마 라인업 8편 7인의 부활 강매강 인사하는 사이 열혈사제2 ...,잇 도 연기 대상 재벌 형사 커넥션 매력 강력반 강 매강 인사 사이 남녀 시상 기대...,잇 도 연기 대상 재벌 형사 커넥션 번 안보현 박지현 재벌 형사 니 중 마이 데몬,인 부활 강매 인사 사이 열혈 사제 시기
2,20240416,안녕하세요 리을입니다 오늘은 2024년 4월 1일부터 4월 15일까지 방송된 드라마...,오늘은 b2024년b 4월 1일부터 4월 15일까지 방송된 b드라마b 시청률 순위를...,2024년 4월 드라마 시청률 순위와 5월 방영 공개 예정 작품 살펴 보기,안녕하세요 리을 은 되 주말극 일일 극 제외 위 토 회 토 오후 분 수호 홍예 김민...,은 되 주말극 일일 극 제외 정작,NaN
3,20240430,편성은 언제나 변경될 수 있으며 이 포스팅은 을 기준으로 작성된 이후 수정 예정이 ...,영상 httpsyoutubewo2JKDWp3lMsiZOlWQLwm0deavd87 5...,공지 2024년 5월 방영할 한국 드라마 라인업모음정리,변경 수 포스팅 기준 작성 되 이후 수정 중 편성 회차 미 표기 경우 일일 제외 단...,주말 졸업 제공 티빙 넷플릭스 정려원 사교육 시장 대명사 수,공지 한국
4,20240501,moonsol 2024년 5월 방영예정드라마 한국드라마 일 월 화 수 목 금 토 1...,moonsol b2024년b 5월 b방영b예정b드라마b 한국b드라마b 일 월 화 수...,2024년 5월 방영 예정 한국 드라마 11,한국 화 수 목 금 토 비밀 히어로 용 수정 졸업 크래시 주 삼식이 삼촌 우리집 커...,한국 화 수 목 금 토 비밀 히어로 용 수정 졸업 크래시 주 삼식이 삼촌,한국


In [3]:
# 결측치 제거
df = df.dropna(subset=['preprocessed_body'])
df.shape

(1947, 7)

In [4]:
# 상위 (max_features)개의 단어에 대해서만 vectorize  -> 각각의 문서들을 벡터 형태로 만들기 위해서 사용
# max_df=0.5: 문서의 절반 이상에서 나타나는 단어를 무시 -> 자주 등장하지만 정보량이 적은 단어 
vectorizer = TfidfVectorizer(max_features=300, max_df=0.5, smooth_idf=True) 
X = vectorizer.fit_transform(df['preprocessed_body'])

In [5]:
print(X.shape)

(1947, 300)


In [6]:
# X는 sparse matrix. 
# 효율적인 저장 방식인 CSR(Compressed Sparse Row) matrix 형식으로 저장되어 있음
X

<1947x300 sparse matrix of type '<class 'numpy.float64'>'
	with 60813 stored elements in Compressed Sparse Row format>

- <span style = 'font-size:1.1em;line-height:1.5em'>TF-IDF의 각 차원의 의미</span>

In [7]:
terms = vectorizer.get_feature_names_out()
print(terms) # 벡터의 각 차원이 어떤 단어를 의미하는지 
print(len(terms)) # 총 벡터의 차원은 300차원 

['가능' '가정' '가족' '가지' '감사' '감상' '감정' '강사' '개인' '거짓말' '검사' '게임' '결혼' '경찰'
 '경찰서' '공감' '공동' '공조' '과거' '과정' '관련' '관심' '국가' '국내' '궁금' '권력' '그룹' '금수저'
 '금요일' '기간' '기대작' '기록' '기사' '기억' '기획' '김태리' '나라' '나의' '나이' '날짜' '남녀' '남편'
 '내용' '네이버' '넷플릭스' '년대' '눈물' '느낌' '능력' '다양' '닥터' '대사' '대상' '대신' '대표' '대학'
 '대한민국' '도시' '동명' '동생' '동시' '드래곤' '등급' '등장' '디즈니' '때문' '라이프' '러브' '로맨스'
 '로맨틱' '리뷰' '링크' '마우스' '마을' '마음' '마이' '마지막' '막내아들' '만화' '맞선' '매력' '매혹'
 '멜로' '모범' '모습' '문제' '미국' '미래' '미스터리' '미정' '바이두' '방송사' '배경' '백과' '범죄' '법정'
 '변호사' '변화' '병원' '보이' '복귀' '복수' '부모' '부부' '부분' '분위기' '비밀' '사건' '사고' '사극'
 '사내' '사실' '사이' '사장' '사회' '살인' '상황' '생활' '선수' '선택' '설정' '성격' '성공' '성장'
 '세계' '세상' '세작' '소녀' '소년' '소리' '소설' '소식' '소재' '수사' '수사반장' '수상' '수요일' '순간'
 '스릴러' '스캔들' '스타' '스토리' '스튜디오' '스트리밍' '슬럼프' '시대' '시리즈' '시절' '시청자' '아내'
 '아들' '아래' '아버지' '아이돌' '아이치' '악귀' '액션' '얼굴' '엄마' '에피소드' '엔터테인먼트' '여성' '여왕'
 '역할' '연기' '연애' '연인' '열혈' '영상' '영화' '예고' '예고편' '예능' '예상' '오늘' '오랜만' '오리지널'
 '오피스' '오후' '올해' '완벽' '외모' '요즘' '욕망' '우영우' '우정' '우

- <span style = 'font-size:1.1em;line-height:1.5em'>TF-IDF matrix에 Truncated SVD를 적용하여 LSA 수행</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'>n_components = 토픽 수</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'>일반적으로 TF-IDF 특징 수의 5%에서 20% 사이의 값을 토픽 수로 설정

In [8]:
model = TruncatedSVD(n_components=30, n_iter=100, random_state=42) 
model.fit(X)

TruncatedSVD(n_components=30, n_iter=100, random_state=42)

$$X=U \Sigma V^{T} \approx U_{trunc} \Sigma_{trunc} V_{trunc}^{T}$$ 
- <span style = 'font-size:1.2em;line-height:1.5em'>여기서 <code>result=model.transform(X)</code>: $U_{trunc} \Sigma_{trunc}$</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'>1947개의 문서를 300차원의 벡터로 (즉, 300개 단어의 조합으로) 표현</span>
    - <span style = 'font-size:1.1em;line-height:1.5em'>그러나, SVD를 활용하여 1947개의 문서를 30차원의 벡터로 (즉, 30개 주제의 조합으로) 표현 </span>

In [9]:
result = model.transform(X)
print(result.shape)

(1947, 30)


In [10]:
result.shape

(1947, 30)

In [11]:
# 30개의 각 topic이 어떻게 300개의 단어 조합으로 되어있는지
print(model.components_.shape)
print(model.components_)

(30, 300)
[[ 0.04889753  0.04296676  0.10717086 ...  0.02863186  0.0378328
   0.03696068]
 [-0.02399007  0.03080954 -0.00619207 ... -0.01428428 -0.02476265
   0.01098491]
 [ 0.01012673 -0.0300276  -0.00837933 ...  0.01252104 -0.01017486
   0.01022944]
 ...
 [-0.02412795  0.02068298 -0.05111321 ...  0.01196771  0.08683914
  -0.01512652]
 [-0.00773854  0.04817673 -0.0495343  ... -0.01085802 -0.08770395
  -0.01731725]
 [ 0.02982678  0.03417488  0.11902947 ...  0.00651792  0.15313194
  -0.00770568]]


In [12]:
terms = vectorizer.get_feature_names_out()
print(terms) # 벡터의 각 차원이 어떤 단어를 의미하는지 (Each dimension is matched with which word)
print(len(terms)) # 총 벡터의 차원은 300차원 (Dimension: 300)

['가능' '가정' '가족' '가지' '감사' '감상' '감정' '강사' '개인' '거짓말' '검사' '게임' '결혼' '경찰'
 '경찰서' '공감' '공동' '공조' '과거' '과정' '관련' '관심' '국가' '국내' '궁금' '권력' '그룹' '금수저'
 '금요일' '기간' '기대작' '기록' '기사' '기억' '기획' '김태리' '나라' '나의' '나이' '날짜' '남녀' '남편'
 '내용' '네이버' '넷플릭스' '년대' '눈물' '느낌' '능력' '다양' '닥터' '대사' '대상' '대신' '대표' '대학'
 '대한민국' '도시' '동명' '동생' '동시' '드래곤' '등급' '등장' '디즈니' '때문' '라이프' '러브' '로맨스'
 '로맨틱' '리뷰' '링크' '마우스' '마을' '마음' '마이' '마지막' '막내아들' '만화' '맞선' '매력' '매혹'
 '멜로' '모범' '모습' '문제' '미국' '미래' '미스터리' '미정' '바이두' '방송사' '배경' '백과' '범죄' '법정'
 '변호사' '변화' '병원' '보이' '복귀' '복수' '부모' '부부' '부분' '분위기' '비밀' '사건' '사고' '사극'
 '사내' '사실' '사이' '사장' '사회' '살인' '상황' '생활' '선수' '선택' '설정' '성격' '성공' '성장'
 '세계' '세상' '세작' '소녀' '소년' '소리' '소설' '소식' '소재' '수사' '수사반장' '수상' '수요일' '순간'
 '스릴러' '스캔들' '스타' '스토리' '스튜디오' '스트리밍' '슬럼프' '시대' '시리즈' '시절' '시청자' '아내'
 '아들' '아래' '아버지' '아이돌' '아이치' '악귀' '액션' '얼굴' '엄마' '에피소드' '엔터테인먼트' '여성' '여왕'
 '역할' '연기' '연애' '연인' '열혈' '영상' '영화' '예고' '예고편' '예능' '예상' '오늘' '오랜만' '오리지널'
 '오피스' '오후' '올해' '완벽' '외모' '요즘' '욕망' '우영우' '우정' '우

In [13]:
model.components_[0].shape

(300,)

- <span style = 'font-size:1.2em;line-height:1.5em'>각 topic에 대해서 상위 5개의 큰 값을 갖는 단어를 출력하는 함수</span>

In [14]:
def get_keyword_by_topic(components, feature_name, n=5):
    for idx, topic in enumerate(components):
        sorted_keyword_idx = np.argsort(components[idx])[-1:-n-1:-1]
        sorted_keyword_result = [(feature_name[x], topic[x].round(4)) for x in sorted_keyword_idx]
        print(f'Topic {idx}: {sorted_keyword_result}')

In [15]:
get_keyword_by_topic(model.components_, terms)

Topic 0: [('로맨스', 0.2339), ('넷플릭스', 0.1871), ('웹툰', 0.156), ('중국', 0.1492), ('웨이보', 0.1487)]
Topic 1: [('텐센트', 0.4871), ('웨이보', 0.4394), ('바이두', 0.2946), ('중국', 0.2694), ('백과', 0.2667)]
Topic 2: [('중국', 0.8221), ('아이치', 0.2162), ('링크', 0.1103), ('아래', 0.1041), ('리뷰', 0.0866)]
Topic 3: [('텐센트', 0.6), ('중국', 0.2204), ('넷플릭스', 0.2177), ('웹툰', 0.1476), ('티빙', 0.0866)]
Topic 4: [('넷플릭스', 0.507), ('웹툰', 0.2721), ('바이두', 0.2023), ('백과', 0.2), ('디즈니', 0.1659)]
Topic 5: [('변호사', 0.4665), ('우영우', 0.1743), ('사건', 0.1654), ('검사', 0.1606), ('넷플릭스', 0.1515)]
Topic 6: [('수사반장', 0.3371), ('형사', 0.3081), ('디즈니', 0.2201), ('플러스', 0.1757), ('범죄', 0.1615)]
Topic 7: [('일본', 0.61), ('게임', 0.1202), ('채널', 0.1104), ('결혼', 0.0941), ('때문', 0.0907)]
Topic 8: [('웹툰', 0.4163), ('형사', 0.3135), ('수사반장', 0.2222), ('일본', 0.1947), ('수사', 0.1856)]
Topic 9: [('티빙', 0.3741), ('디즈니', 0.3136), ('플러스', 0.2667), ('이미지', 0.1942), ('웨이브', 0.1712)]
Topic 10: [('로맨스', 0.3507), ('청춘', 0.2768), ('연애', 0.2407), ('소년', 0.1438), ('판타지

# <b> 2. LDA(Latent Dirichlet Allocation)</b> 

In [16]:
# preprocessed_body에 있는 단어를 리스트에 넣은 칼럼 생성
df['word_list'] = df['preprocessed_body'].apply(lambda x: x.split())

In [17]:
df['word_list'].value_counts()

word_list
[본문, 수]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     93
[상기, 일정, 편성, 방송사, 사정, 변경, 수, 수, 김양희, 박, 진리, 강소라, 장승조, 이혼, 이별, 이혼, 전문, 변호사, 인생, 성장기, 수, 목, 딜리버리, 맨, 강, 솔, 박대희, 주효진, 박혜영, 보경, 윤찬영, 방, 민아, 김민석, 생계, 형, 택시, 기사, 영민, 기억상실, 영혼, 지현, 환장, 듀오, 세상, 하이텐션, 수, 사극, 종이달, 김승우, 노윤수, 김서형, 유선, 서영희, 이천희, 공정환, 결핍, 욕망, 편안, 삶, 유이화, 은행, 계약, 직, 사원, 던, 중, 고객, 돈, 손, 일상, 수, 스릴러, 가쿠다, 미쓰요, 소설, 종이달, 데보라, 이태곤, 아경, 유인나, 윤현민, 주상, 연애, 코치, 데보라, 마성, 매력, 지니, 남, 수혁, 에피소드, 코, 연애, 진심, 데보라, 변경, 오랫동안, 당신을, 기다렸습니다, ...]                                                2
[은, 평소, 중국, 설명, 치아, 문, 단순, 적소, 미

In [18]:
lst = df['word_list'].apply(lambda words: [word for word in words if len(word) == 1])
lst
# word_list에서 한 글자인 단어 확인 -> 무의미하다고 판단

0       [은, 문, 미, 호, 폭, 부, 작, 환, 주, 삶, 우, 우, 수, 합, 지, ...
1       [잇, 도, 강, 편, 편, 것, 번, 니, 중, 되, 작, 팀, 돈, 돈, 빽, ...
2       [은, 되, 극, 위, 토, 회, 토, 분, 되, 회, 위, 회, 월, 화, 시, ...
3       [수, 되, 중, 미, 수, 부, 작, 주, 봄, 되, 송, 날, 후, 수, 목, ...
4       [화, 수, 목, 금, 토, 용, 주, 되, 수, 수, 목, 주, 키, 송, 되, ...
                              ...                        
1942    [중, 드, 행, 후, 종, 꽃, 옥, 화, 금, 시, 회, 니, 중, 드, 남, ...
1943    [함, 정, 맡, 극, 니, 중, 데, 중, 게, 지, 일, 분, 시, 분, 분, ...
1944    [비, 날, 집, 콕, 꿀, 잼, 정, 편, 편, 중, 박, 지, 기, 라, 쇼, ...
1945    [새, 지, 폭, 업, 안, 살, 편, 업, 토, 미, 토, 시, 분, 토, 뒤, ...
1946    [금, 토, 빅, 군, 후, 작, 도, 박, 수, 빅, 뜻, 금, 토, 임, 빅, ...
Name: word_list, Length: 1947, dtype: object

In [19]:
# word_list에서 한 글자인 단어 삭제
df['word_list'] = df['word_list'].apply(lambda words: [word for word in words if len(word) > 1])
df['word_list'].value_counts()

word_list
[본문]                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         93
[상기, 일정, 편성, 방송사, 사정, 변경, 김양희, 진리, 강소라, 장승조, 이혼, 이별, 이혼, 전문, 변호사, 인생, 성장기, 딜리버리, 박대희, 주효진, 박혜영, 보경, 윤찬영, 민아, 김민석, 생계, 택시, 기사, 영민, 기억상실, 영혼, 지현, 환장, 듀오, 세상, 하이텐션, 사극, 종이달, 김승우, 노윤수, 김서형, 유선, 서영희, 이천희, 공정환, 결핍, 욕망, 편안, 유이화, 은행, 계약, 사원, 고객, 일상, 스릴러, 가쿠다, 미쓰요, 소설, 종이달, 데보라, 이태곤, 아경, 유인나, 윤현민, 주상, 연애, 코치, 데보라, 마성, 매력, 지니, 수혁, 에피소드, 연애, 진심, 데보라, 변경, 오랫동안, 당신을, 기다렸습니다, 철수, 나인우, 김지은, 이규, 권율, 시골, 마초, 형사, 미스터리, 추적, 영심, 김경, 전선, 송하윤, 동해, 인생, 희로애락, 영심, 경태, 재회, ...]                      

In [20]:
from gensim import corpora

word_dict = corpora.Dictionary(df['word_list'])

In [21]:
corpus = [word_dict.doc2bow(text) for text in df['word_list']]

In [22]:
len(corpus)

1947

In [23]:
corpus[2]  ## 2번째 문서에 대해 각각의 단어들이 몇 번을 나왔는지.. 
## -> (1, 1): 1번 단어가 1번 나오고... 
## -> (4, 3): 4번 단어가 3번 나오고...

[(0, 1),
 (5, 1),
 (16, 1),
 (46, 1),
 (52, 1),
 (62, 2),
 (90, 1),
 (91, 1),
 (113, 1),
 (122, 1),
 (146, 1),
 (147, 2),
 (171, 1),
 (187, 1),
 (192, 1),
 (193, 1),
 (195, 1),
 (198, 3),
 (206, 1),
 (211, 2),
 (220, 1),
 (222, 1),
 (230, 1),
 (233, 1),
 (240, 3),
 (241, 3),
 (260, 1),
 (263, 1),
 (265, 1),
 (272, 1),
 (283, 1),
 (284, 1),
 (291, 1),
 (302, 1),
 (305, 1),
 (311, 1),
 (312, 1),
 (313, 1),
 (314, 1),
 (315, 3),
 (316, 1),
 (317, 1),
 (318, 1),
 (319, 1),
 (320, 1),
 (321, 1),
 (322, 1),
 (323, 3),
 (324, 1),
 (325, 1),
 (326, 1),
 (327, 1),
 (328, 6),
 (329, 1),
 (330, 1),
 (331, 1),
 (332, 1),
 (333, 1),
 (334, 1),
 (335, 1),
 (336, 1),
 (337, 1),
 (338, 2),
 (339, 1),
 (340, 1),
 (341, 1),
 (342, 1),
 (343, 1),
 (344, 1),
 (345, 1),
 (346, 2),
 (347, 5),
 (348, 1),
 (349, 1),
 (350, 2),
 (351, 2),
 (352, 1),
 (353, 1),
 (354, 1),
 (355, 1),
 (356, 1),
 (357, 1),
 (358, 4),
 (359, 1),
 (360, 1),
 (361, 1),
 (362, 1),
 (363, 1),
 (364, 1),
 (365, 2),
 (366, 1),
 (367, 1)

In [24]:
[(idx,word) for idx, word in word_dict.items()]
## 0번 단어는 가족...
## 1번 단어는 가지...

[(0, '가족'),
 (1, '가지'),
 (2, '감성'),
 (3, '개인'),
 (4, '거짓말'),
 (5, '결심'),
 (6, '결혼식'),
 (7, '고교'),
 (8, '고려'),
 (9, '고심'),
 (10, '고유'),
 (11, '고향'),
 (12, '관련'),
 (13, '관심'),
 (14, '괴롭힘'),
 (15, '궁금'),
 (16, '기대작'),
 (17, '기회'),
 (18, '남녀'),
 (19, '내용'),
 (20, '노력'),
 (21, '단순'),
 (22, '대만'),
 (23, '대학'),
 (24, '댓글'),
 (25, '덕분'),
 (26, '도움'),
 (27, '동거'),
 (28, '동급'),
 (29, '동아리'),
 (30, '동창'),
 (31, '두뇌'),
 (32, '때문'),
 (33, '리프'),
 (34, '마음'),
 (35, '만성'),
 (36, '만화'),
 (37, '망치'),
 (38, '명승'),
 (39, '무력'),
 (40, '문제'),
 (41, '방지'),
 (42, '배려'),
 (43, '백샤오둑이'),
 (44, '보호'),
 (45, '빌리'),
 (46, '사건'),
 (47, '사망'),
 (48, '사이트'),
 (49, '사촌'),
 (50, '상연'),
 (51, '상처'),
 (52, '상황'),
 (53, '색감'),
 (54, '생기'),
 (55, '샹견'),
 (56, '선택'),
 (57, '설명'),
 (58, '성도'),
 (59, '세상'),
 (60, '소녀'),
 (61, '소문'),
 (62, '소설'),
 (63, '손해'),
 (64, '순서'),
 (65, '스토리'),
 (66, '스포'),
 (67, '스포일러'),
 (68, '시운'),
 (69, '아쉬움'),
 (70, '언니'),
 (71, '언성'),
 (72, '얼굴'),
 (73, '업로드'),
 (74, '여주인공'),
 (75, '여친'),
 (76, 

## LDA 모델 훈련 (Topic 수: 30개로 설정)

In [25]:
import gensim

In [26]:
N_TOPICS = 30
ldamodel = gensim.models.ldamodel.LdaModel(corpus, 
                                           num_topics = N_TOPICS,  ## 주제 몇 개 할 건지
                                           id2word=word_dict)   ## vocabulary set

### 각 topic의 대표 단어들을 출력

In [27]:
topics = ldamodel.print_topics(num_words=5)  ## 각 토픽들에 대해서 몇 개의 단어(num_words)로 표현할 것인지
## 대표적인 단어를 알 수 있음
for topic in topics:
    print(topic)

(28, '0.005*"로맨스" + 0.004*"넷플릭스" + 0.004*"웹툰" + 0.003*"변호사" + 0.003*"비밀"')
(8, '0.012*"중국" + 0.006*"여왕" + 0.005*"로맨스" + 0.004*"눈물" + 0.004*"티빙"')
(6, '0.008*"로맨스" + 0.006*"넷플릭스" + 0.005*"소설" + 0.004*"웹툰" + 0.003*"코미디"')
(27, '0.008*"로맨스" + 0.005*"티빙" + 0.004*"넷플릭스" + 0.004*"웹툰" + 0.003*"내용"')
(29, '0.006*"중국" + 0.005*"로맨스" + 0.004*"넷플릭스" + 0.004*"한국" + 0.003*"판타지"')
(25, '0.009*"변호사" + 0.004*"티빙" + 0.004*"로맨스" + 0.003*"웹툰" + 0.003*"가족"')
(15, '0.005*"로맨스" + 0.005*"한국" + 0.004*"형사" + 0.003*"인생" + 0.003*"복수"')
(0, '0.006*"웹툰" + 0.005*"로맨스" + 0.004*"형사" + 0.004*"한국" + 0.003*"인생"')
(19, '0.008*"넷플릭스" + 0.006*"로맨스" + 0.005*"디즈니" + 0.004*"티빙" + 0.003*"플러스"')
(7, '0.011*"웹툰" + 0.005*"로맨스" + 0.005*"넷플릭스" + 0.003*"판타지" + 0.003*"티빙"')
(20, '0.013*"넷플릭스" + 0.007*"웹툰" + 0.005*"로맨스" + 0.003*"한국" + 0.003*"디즈니"')
(22, '0.007*"로맨스" + 0.006*"웹툰" + 0.005*"넷플릭스" + 0.004*"인생" + 0.003*"편성"')
(16, '0.005*"로맨스" + 0.005*"사제" + 0.005*"넷플릭스" + 0.004*"웹툰" + 0.004*"열혈"')
(1, '0.008*"넷플릭스" + 0.006*"일본" + 0.006*"게임

## LDA 시각화

In [28]:
from IPython.core.display import HTML
import pyLDAvis
from pyLDAvis import gensim_models

In [29]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(ldamodel, corpus, word_dict)
pyLDAvis.display(vis)

## 문서 별 Topic 분포

In [30]:
for i, topic_list in enumerate(ldamodel[corpus]):
    print(f'{i}번째 문서의 topic 비중: {topic_list}')
    if i==5:
        break

0번째 문서의 topic 비중: [(4, 0.34472203), (9, 0.37885332), (19, 0.16260523), (29, 0.10974763)]
1번째 문서의 topic 비중: [(6, 0.10598164), (9, 0.027258838), (14, 0.11199404), (15, 0.20593159), (16, 0.19586477), (21, 0.2878771), (27, 0.06235851)]
2번째 문서의 topic 비중: [(8, 0.88860625), (12, 0.10890996)]
3번째 문서의 topic 비중: [(6, 0.48678643), (20, 0.014084119), (21, 0.0269613), (27, 0.46852043)]
4번째 문서의 topic 비중: [(6, 0.99781334)]
5번째 문서의 topic 비중: [(2, 0.94782543), (9, 0.04967918)]


### 각 문서에 대해서 Topic의 비중을 나타내는 함수 작성

In [31]:
## 교수님 코드 

def get_topic_ratio_for_each_document(ldamodel, corpus):
    results = []
    for i, topic_list in enumerate(ldamodel[corpus]):
        sorted_topic_list = sorted(topic_list, key = lambda x: x[1], reverse=True)
        most_important_topic, most_important_ratio = sorted_topic_list[0]
        doc_result = [i, most_important_topic, most_important_ratio, sorted_topic_list]
        results.append(doc_result)
        
    results = pd.DataFrame(results, columns = ['문서 번호', '가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중'])
    return results

In [32]:
df_results = get_topic_ratio_for_each_document(ldamodel, corpus)

df_results

,문서 번호,가장 비중이 높은 토픽,가장 높은 토픽의 비중,각 토픽의 비중
0,0,9,0.378985,"[(9, 0.37898505), (4, 0.344172), (19, 0.166061..."
1,1,21,0.283923,"[(21, 0.28392264), (15, 0.20673802), (16, 0.19..."
2,2,8,0.890643,"[(8, 0.89064336), (12, 0.106872804)]"
3,3,6,0.484916,"[(6, 0.48491606), (27, 0.4682014), (21, 0.0218..."
4,4,6,0.997813,"[(6, 0.99781334)]"
...,...,...,...,...
1942,1942,27,0.996658,"[(27, 0.996658)]"
1943,1943,6,0.996921,"[(6, 0.99692065)]"
1944,1944,12,0.996095,"[(12, 0.99609536)]"
1945,1945,20,0.939938,"[(20, 0.9399377), (24, 0.03622668), (13, 0.018..."


In [33]:
## 참고 사이트: https://wikidocs.net/30708

def make_topictable_per_doc(ldamodel, corpus):
    # 데이터를 저장할 빈 리스트 초기화
    topic_data = []

    # 몇 번째 문서인지를 의미하는 문서 번호와 해당 문서의 토픽 비중을 한 줄씩 꺼내온다.
    for i, topic_list in enumerate(ldamodel[corpus]):
        doc = topic_list[0] if ldamodel.per_word_topics else topic_list            
        doc = sorted(doc, key=lambda x: (x[1]), reverse=True)
        # 각 문서에 대해서 비중이 높은 토픽순으로 토픽을 정렬한다.
        # EX) 정렬 전 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (10번 토픽, 5%), (12번 토픽, 21.5%), 
        # Ex) 정렬 후 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (12번 토픽, 21.5%), (10번 토픽, 5%)
        # 48 > 25 > 21 > 5 순으로 정렬이 된 것.

        # 모든 문서에 대해서 각각 아래를 수행
        for j, (topic_num, prop_topic) in enumerate(doc): #  몇 번 토픽인지와 비중을 나눠서 저장한다.
            if j == 0:  # 정렬을 한 상태이므로 가장 앞에 있는 것이 가장 비중이 높은 토픽
                topic_data.append([i, int(topic_num), round(prop_topic, 4), topic_list])
                # 가장 비중이 높은 토픽과, 가장 비중이 높은 토픽의 비중과, 전체 토픽의 비중을 저장한다.
            else:
                break
            
    # 리스트를 데이터프레임으로 변환
    topic_table = pd.DataFrame(topic_data, columns=['문서 번호', '가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중'])

    return(topic_table)

In [34]:
topictable = make_topictable_per_doc(ldamodel, corpus)
topictable = topictable.reset_index() # 문서 번호을 의미하는 열(column)로 사용하기 위해서 인덱스 열을 하나 더 만든다.
topictable[:10]

,index,문서 번호,가장 비중이 높은 토픽,가장 높은 토픽의 비중,각 토픽의 비중
0,0,0,9,0.3831,"[(4, 0.34834167), (9, 0.3830672), (19, 0.15880..."
1,1,1,21,0.2845,"[(6, 0.1082582), (9, 0.028176207), (14, 0.1159..."
2,2,2,8,0.8894,"[(8, 0.889356), (12, 0.10816018)]"
3,3,3,6,0.4931,"[(6, 0.49307945), (21, 0.026943203), (27, 0.47..."
4,4,4,6,0.9978,"[(6, 0.99781334)]"
5,5,5,2,0.9478,"[(2, 0.94784385), (9, 0.049660757)]"
6,6,6,9,0.4416,"[(6, 0.29778445), (9, 0.44156283), (10, 0.2351..."
7,7,7,9,0.6297,"[(0, 0.15741904), (9, 0.6297167), (13, 0.18142..."
8,8,8,7,0.8685,"[(7, 0.86846805), (22, 0.011783529), (24, 0.11..."
9,9,9,21,0.6396,"[(1, 0.09635396), (18, 0.1383771), (21, 0.6395..."


In [35]:
## 교수님 코드 + 각 토픽 별 키워드를 볼 수 있는 함수 (GPT의 도움을 조금 받음!)


def get_topic_keywords(ldamodel, num_keywords=5):
    topic_keywords = {}
    for topic_id in range(ldamodel.num_topics):
        keywords = [word for word, _ in ldamodel.show_topic(topic_id, topn=num_keywords)]
        topic_keywords[topic_id] = keywords
    return topic_keywords

def get_topic_ratio_for_each_document(ldamodel, corpus):
    topic_keywords = get_topic_keywords(ldamodel)
    results = []
    for i, topic_list in enumerate(ldamodel[corpus]):
        sorted_topic_list = sorted(topic_list, key = lambda x: x[1], reverse=True)
        most_important_topic, most_important_ratio = sorted_topic_list[0]
        second_most_important_topic, second_most_important_ratio = sorted_topic_list[1] if len(sorted_topic_list) > 1 else (None, 0)

        keywords_1 = ", ".join(topic_keywords[most_important_topic])
        keywords_2 = ", ".join(topic_keywords[second_most_important_topic]) if second_most_important_topic is not None else ''
        doc_result = [i, most_important_topic, most_important_ratio, sorted_topic_list, keywords_1, keywords_2]
        results.append(doc_result)
        
    results = pd.DataFrame(results, columns=['문서 번호', '가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중', 'Top1_Topic_keyword', 'Top2_Topic_keyword'])
    return results

In [36]:
df_results = get_topic_ratio_for_each_document(ldamodel, corpus)

df_results

,문서 번호,가장 비중이 높은 토픽,가장 높은 토픽의 비중,각 토픽의 비중,Top1_Topic_keyword,Top2_Topic_keyword
0,0,9,0.380131,"[(9, 0.3801307), (4, 0.34639984), (19, 0.16146...","티빙, 로맨스, 사극, 플러스, 역할","웹툰, 넷플릭스, 내용, 로맨스, 중국"
1,1,21,0.288475,"[(21, 0.2884753), (15, 0.20707276), (16, 0.195...","로맨스, 넷플릭스, 티빙, 인생, 판타지","로맨스, 한국, 형사, 인생, 복수"
2,2,8,0.888729,"[(8, 0.88872933), (12, 0.10878688)]","중국, 여왕, 로맨스, 눈물, 티빙","넷플릭스, 로맨스, 스릴러, 한국, 남편"
3,3,6,0.481740,"[(6, 0.48173967), (27, 0.4613045), (21, 0.0310...","로맨스, 넷플릭스, 소설, 웹툰, 코미디","로맨스, 티빙, 넷플릭스, 웹툰, 내용"
4,4,6,0.997813,"[(6, 0.99781334)]","로맨스, 넷플릭스, 소설, 웹툰, 코미디",
...,...,...,...,...,...,...
1942,1942,27,0.996658,"[(27, 0.996658)]","로맨스, 티빙, 넷플릭스, 웹툰, 내용",
1943,1943,6,0.996921,"[(6, 0.99692065)]","로맨스, 넷플릭스, 소설, 웹툰, 코미디",
1944,1944,12,0.996095,"[(12, 0.99609536)]","넷플릭스, 로맨스, 스릴러, 한국, 남편",
1945,1945,20,0.949286,"[(20, 0.94928616), (24, 0.035571363)]","넷플릭스, 웹툰, 로맨스, 한국, 디즈니","넷플릭스, 로맨스, 티빙, 웹툰, 복수"
